In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [1]:
%%capture
!pip install modelscope==1.4.2
!pip install open_clip_torch
!pip install pytorch-lightning

In [2]:
cd /content/gdrive/MyDrive/modelscope

/content/gdrive/MyDrive/modelscope


In [3]:
# Download weights
import pathlib
from huggingface_hub import snapshot_download
from modelscope.outputs import OutputKeys

model_dir = pathlib.Path('weights')
snapshot_download('damo-vilab/modelscope-damo-text-to-video-synthesis',
                   repo_type='model', local_dir=model_dir)

Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

'/content/gdrive/MyDrive/modelscope/weights'

In [4]:
# Construct pipeline
import pathlib
from modelscope.pipelines import pipeline
model_dir = pathlib.Path('weights')
pipe = pipeline('text-to-video-synthesis', model_dir.as_posix())

2023-04-16 13:55:49,726 - modelscope - INFO - PyTorch version 2.0.0+cu118 Found.
2023-04-16 13:55:49,737 - modelscope - INFO - TensorFlow version 2.12.0 Found.
2023-04-16 13:55:49,739 - modelscope - INFO - Loading ast index from /root/.cache/modelscope/ast_indexer
2023-04-16 13:55:49,858 - modelscope - INFO - Loading done! Current index file version is 1.4.2, with md5 fe2ecfee8d0e1b5f9e276d73bc799abb and a total number of 842 components indexed
2023-04-16 13:56:03,784 - modelscope - INFO - initiate model from weights
2023-04-16 13:56:03,788 - modelscope - INFO - initiate model from location weights.
2023-04-16 13:56:03,795 - modelscope - INFO - initialize model from weights
2023-04-16 13:57:44,108 - modelscope - WARNING - No preprocessor field found in cfg.
2023-04-16 13:57:44,111 - modelscope - WARNING - No val key and type key found in preprocessor domain of configuration.json file.
2023-04-16 13:57:44,112 - modelscope - WARNING - Cannot find available config to build preprocessor at

In [ ]:
# Test Rendering
test_text = {'text': 'A panda eating bamboo on a rock.',}
output_video_path = pipe(test_text,)['output_video']
print('output_video_path:', output_video_path)

2023-04-11 13:46:38,913 - modelscope - WARNING - task text-to-video-synthesis input definition is missing
2023-04-11 13:47:51,618 - modelscope - WARNING - task text-to-video-synthesis output keys are missing


output_video_path: /tmp/tmpz64flgi1.mp4


In [ ]:
!mv /tmp/tmpkupqfeh9.mp4 /content/gdrive/MyDrive/modelscope/output/

In [5]:
drei_spinnerinnen = "There was once a girl who was lazy and would not spin, and her mother could not persuade her to it, do what she would. At last the mother became angry and out of patience, and gave her a good beating, so that she cried out loudly. At that moment the Queen was going by; as she heard the crying, she stopped; and, going into the house, she asked the mother why she was beating her daughter, so that every one outside in the street could hear her cries. The woman was ashamed to tell of her daughter's laziness, so she said, 'I cannot stop her from spinning; she is for ever at it, and I am poor and cannot furnish her with flax enough.' Then the Queen answered, 'I like nothing better than the sound of the spinning-wheel, and always feel happy when I hear its humming; let me take your daughter with me to the castle - I have plenty of flax, she shall spin there to her heart's content.' The mother was only too glad of the offer, and the Queen took the girl with her."

In [12]:
def split_text_into_prompts(txt, repeats_per_prompt=1):
    prompts = []
    s = 0
    for i in range(len(txt) - 1):
        if txt[i] == ".":
            if txt[i+1] == "'":
                e = i + 2
            e = i + 1
            for j in range(repeats_per_prompt):
                prompts.append(txt[s:e])
            s = e + 1
    return prompts

In [ ]:
prompts = split_text_into_prompts(drei_spinnerinnen)

In [ ]:
prompts

['There was once a girl who was lazy and would not spin, and her mother could not persuade her to it, do what she would.',
 'At last the mother became angry and out of patience, and gave her a good beating, so that she cried out loudly.',
 'At that moment the Queen was going by; as she heard the crying, she stopped; and, going into the house, she asked the mother why she was beating her daughter, so that every one outside in the street could hear her cries.',
 "The woman was ashamed to tell of her daughter's laziness, so she said, 'I cannot stop her from spinning; she is for ever at it, and I am poor and cannot furnish her with flax enough.",
 " Then the Queen answered, 'I like nothing better than the sound of the spinning-wheel, and always feel happy when I hear its humming; let me take your daughter with me to the castle - I have plenty of flax, she shall spin there to her heart's content."]

In [7]:
import os
from shutil import move
outpath = "/content/gdrive/MyDrive/modelscope/output/"

def synthesise_story(prompts):
    for i, prompt in enumerate(prompts):
        test_text = {'text': prompt,}
        output_video_path = pipe(test_text,)['output_video']
        print('output_video_path:', output_video_path)
        move(output_video_path, outpath + str(i)+'.mp4')

In [11]:
synthesise_story(prompts)

NameError: ignored

In [8]:
%%capture
!pip install moviepy

In [9]:
from moviepy.editor import *

def concatenate(video_clip_paths, output_path, method="compose"):
    """Concatenates several video files into one video file
    and save it to `output_path`. Note that extension (mp4, etc.) must be added to `output_path`
    `method` can be either 'compose' or 'reduce':
        `reduce`: Reduce the quality of the video to the lowest quality on the list of `video_clip_paths`.
        `compose`: type help(concatenate_videoclips) for the info"""
    # create VideoFileClip object for each video file
    clips = [VideoFileClip(c) for c in video_clip_paths]
    if method == "reduce":
        # calculate minimum width & height across all clips
        min_height = min([c.h for c in clips])
        min_width = min([c.w for c in clips])
        # resize the videos to the minimum
        clips = [c.resize(newsize=(min_width, min_height)) for c in clips]
        # concatenate the final video
        final_clip = concatenate_videoclips(clips)
    elif method == "compose":
        # concatenate the final video with the compose method provided by moviepy
        final_clip = concatenate_videoclips(clips, method="compose")
    # write the output video file
    final_clip.write_videofile(output_path)

In [ ]:
clips_dir = "/content/gdrive/MyDrive/modelscope/output/"
concatenate([clips_dir+i for i in os.listdir(clips_dir) if i.endswith('mp4')], 'spinnerinnen.mp4')

In [ ]:
prompts = split_text_into_prompts(drei_spinnerinnen, repeats_per_prompt=3)
synthesise_story(prompts)

2023-04-16 13:58:57,924 - modelscope - WARNING - task text-to-video-synthesis input definition is missing
